# Projet 6 : Analysez les ventes d'une librairie avec R ou Python

# Sommaire<a id='sommaire'></a>
<ol><li><a href="#import">Import des librairies</a></li>
    <li><a href="#customers">Fichier customers</a></li> 
    <li><a href="#products">Fichiers products</a></li> 
    <li><a href="#transactions">Fichiers transactions</a></li> 
    <li><a href="#gestion">Gestion des produits et clients sans transaction</a></li> 
    <li><a href="#jointure">Jointure</a></li> 
    <li><a href="#export">Export</a></li> 
</ol>

# Préparation des données

<a id='import' ></a>
## Import des librairies<a href="#sommaire" style="font-size : 10px ; padding-left:10px">Retour Sommaire</a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as snsct
import scipy.stats as st

folder = 'D:/Data_Analyst_Projet/Projet 6/Donnees python/'

## Import des fichiers

In [2]:
customers=pd.read_csv(folder+'customers.csv')
customers.head()

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


In [3]:
products=pd.read_csv(folder+'products.csv')
products.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


In [4]:
transactions=pd.read_csv(folder+'transactions.csv')
transactions.head()

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


<a id='customers' ></a>
## Fichier customers<a href="#sommaire" style="font-size : 10px ; padding-left:10px">Retour Sommaire</a>

In [5]:
customers.head()

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


### Exploration

In [6]:
customers.shape

(8623, 3)

In [7]:
customers.info()
# pas de valeurs manquantes à priori

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


In [8]:
customers.describe(include='all')
# pas de doublons de clients, moyenne et médiane de birth proche, pas de valeurs abberantes à priori

,client_id,sex,birth
count,8623,8623,8623.000000
unique,8623,2,NaN
top,c_4410,f,NaN
freq,1,4491,NaN
mean,NaN,NaN,1978.280877
std,NaN,NaN,16.919535
min,NaN,NaN,1929.000000
25%,NaN,NaN,1966.000000
50%,NaN,NaN,1979.000000
75%,NaN,NaN,1992.000000


### Traitement des valeurs nulles

In [9]:
customers.isna().sum()
# pas de valeurs manquantes comme on avait vu avec info()

client_id    0
sex          0
birth        0
dtype: int64

### Traitement des doublons

In [10]:
customers[customers.duplicated(subset='client_id')]
# pas de doublons comme on avait vu avec describe()

,client_id,sex,birth


### Ajout de la colonne âge

In [11]:
customers['age']=2023 - customers['birth']
customers.head()

,client_id,sex,birth,age
0,c_4410,f,1967,56
1,c_7839,f,1975,48
2,c_1699,f,1984,39
3,c_5961,f,1962,61
4,c_5320,m,1943,80


In [12]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
 3   age        8623 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 269.6+ KB


In [13]:
customers.describe()

,birth,age
count,8623.000000,8623.000000
mean,1978.280877,44.719123
std,16.919535,16.919535
min,1929.000000,19.000000
25%,1966.000000,31.000000
50%,1979.000000,44.000000
75%,1992.000000,57.000000
max,2004.000000,94.000000


### Modification des unités

In [14]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
 3   age        8623 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 269.6+ KB


In [15]:
customers['sex']=customers['sex'].astype('category')

In [16]:
customers.dtypes

client_id      object
sex          category
birth           int64
age             int64
dtype: object

In [17]:
customers.head()

,client_id,sex,birth,age
0,c_4410,f,1967,56
1,c_7839,f,1975,48
2,c_1699,f,1984,39
3,c_5961,f,1962,61
4,c_5320,m,1943,80


<a id='products' ></a>
## Fichier products<a href="#sommaire" style="font-size : 10px ; padding-left:10px">Retour Sommaire</a>

In [18]:
products.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


### Exploration

In [19]:
products.shape

(3287, 3)

In [20]:
products.info()
# pas de valeurs manquantes à priori

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB


In [21]:
products.describe(include='all')
# pas de doublons pour id_product à priori
# le min à -1 --> déterminer la raison

,id_prod,price,categ
count,3287,3287.000000,3287.000000
unique,3287,NaN,NaN
top,0_1421,NaN,NaN
freq,1,NaN,NaN
mean,NaN,21.856641,0.370246
std,NaN,29.847908,0.615387
min,NaN,-1.000000,0.000000
25%,NaN,6.990000,0.000000
50%,NaN,13.060000,0.000000
75%,NaN,22.990000,1.000000


### Traitement des valeurs manquantes

In [22]:
products.isna().sum()
# pas de valeurs manquantes

id_prod    0
price      0
categ      0
dtype: int64

### Traitement des doublons

In [23]:
products.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


In [24]:
products[products.duplicated('id_prod')]
# pas de doublons

,id_prod,price,categ


### Modification des unités

In [25]:
products['categ']=products['categ'].astype('category')

In [26]:
products.dtypes

id_prod      object
price       float64
categ      category
dtype: object

### Traitement des valeurs abberantes

In [27]:
products.sort_values('price')

,id_prod,price,categ
731,T_0,-1.00,0
2355,0_202,0.62,0
2272,0_528,0.62,0
370,0_120,0.66,0
1211,0_1844,0.77,0
...,...,...,...
2779,2_30,233.54,2
1436,2_167,236.99,2
394,2_158,247.22,2
724,2_76,254.44,2


Nous avons un prix égal à -1, on voit que l'id_produit est écrit différement des autres (T_0).

On verra plus tard que ce produit est un produit test et qu'il est à supprimer.

<a id='transactions' ></a>
## Fichier transactions<a href="#sommaire" style="font-size : 10px ; padding-left:10px">Retour Sommaire</a>

In [28]:
transactions.head()

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


### Exploration

In [29]:
transactions.shape

(679532, 4)

In [30]:
transactions.info()
# pas de valeurs manquantes à priori

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679532 entries, 0 to 679531
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     679532 non-null  object
 1   date        679532 non-null  object
 2   session_id  679532 non-null  object
 3   client_id   679532 non-null  object
dtypes: object(4)
memory usage: 20.7+ MB


In [31]:
transactions.describe(include='all')
# 20 produits non vendus (3287 id_products dans products) --> voir lesquels
# date bizarre --> identifier le problème
# 21 clients sans achats --> voir lesquels

,id_prod,date,session_id,client_id
count,679532,679532,679532,679532
unique,3267,679371,342316,8602
top,1_369,test_2021-03-01 02:30:02.237413,s_0,c_1609
freq,2252,13,200,25488


In [32]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   id_prod  3287 non-null   object  
 1   price    3287 non-null   float64 
 2   categ    3287 non-null   category
dtypes: category(1), float64(1), object(1)
memory usage: 54.8+ KB


In [33]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   client_id  8623 non-null   object  
 1   sex        8623 non-null   category
 2   birth      8623 non-null   int64   
 3   age        8623 non-null   int64   
dtypes: category(1), int64(2), object(1)
memory usage: 210.8+ KB


### Traitement des valeurs manquantes

In [34]:
transactions.isna().sum()
# pas de valeurs manquantes

id_prod       0
date          0
session_id    0
client_id     0
dtype: int64

### Identification des anomalies de la colonnes date

In [35]:
transactions[transactions['date'].str.contains('test')]
# On voit que toutes ces transactions sont des tests

,id_prod,date,session_id,client_id
3019,T_0,test_2021-03-01 02:30:02.237419,s_0,ct_0
5138,T_0,test_2021-03-01 02:30:02.237425,s_0,ct_0
9668,T_0,test_2021-03-01 02:30:02.237437,s_0,ct_1
10728,T_0,test_2021-03-01 02:30:02.237436,s_0,ct_0
15292,T_0,test_2021-03-01 02:30:02.237430,s_0,ct_0
...,...,...,...,...
657830,T_0,test_2021-03-01 02:30:02.237417,s_0,ct_0
662081,T_0,test_2021-03-01 02:30:02.237427,s_0,ct_1
670680,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_1
671647,T_0,test_2021-03-01 02:30:02.237424,s_0,ct_1


In [36]:
transactions[transactions['date'].str.contains('test')]['client_id'].unique()
# 2 clients créés pour les tests, on pourra les supprimer de customers pour notre analyse

array(['ct_0', 'ct_1'], dtype=object)

In [37]:
transactions[transactions['date'].str.contains('test')]['id_prod'].unique()
# 1 produit créé pour les tests, on pourra le supprimer de products

array(['T_0'], dtype=object)

### Suppression des anomalies

In [38]:
# Suppression transactions tests
transactions.drop(transactions[transactions['date'].str.contains('test')].index, inplace=True)

In [39]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679332 entries, 0 to 679531
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     679332 non-null  object
 1   date        679332 non-null  object
 2   session_id  679332 non-null  object
 3   client_id   679332 non-null  object
dtypes: object(4)
memory usage: 25.9+ MB


In [40]:
# Suppression du produit test, cela correspond au produit identifié plus haut au prix = -1
products.drop(products[products['id_prod']=='T_0'].index, inplace=True)

In [41]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3286 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   id_prod  3286 non-null   object  
 1   price    3286 non-null   float64 
 2   categ    3286 non-null   category
dtypes: category(1), float64(1), object(1)
memory usage: 80.4+ KB


In [42]:
# Suppression des clients tests
customers.drop(customers[customers['client_id'].isin(['ct_0', 'ct_1'])].index, inplace=True)

In [43]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8621 entries, 0 to 8622
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   client_id  8621 non-null   object  
 1   sex        8621 non-null   category
 2   birth      8621 non-null   int64   
 3   age        8621 non-null   int64   
dtypes: category(1), int64(2), object(1)
memory usage: 277.9+ KB


### Traitement des doublons

In [44]:
transactions[transactions.duplicated()]
# il n'y a plus de doublons

,id_prod,date,session_id,client_id


### Modification des unités

In [45]:
transactions.head()

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


In [46]:
# Modification de la colonne date au type datetime
transactions['date']=pd.to_datetime(transactions['date'], format="%Y %m %d")

In [47]:
transactions.head()

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


In [48]:
# création variable mois
transactions['mois']=transactions['date'].dt.strftime('%Y-%m')
transactions.head()

,id_prod,date,session_id,client_id,mois
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,2022-05
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,2022-02
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,2022-06
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941,2021-06
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232,2023-01


In [49]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679332 entries, 0 to 679531
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     679332 non-null  object        
 1   date        679332 non-null  datetime64[ns]
 2   session_id  679332 non-null  object        
 3   client_id   679332 non-null  object        
 4   mois        679332 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 31.1+ MB


In [50]:
transactions.head()

,id_prod,date,session_id,client_id,mois
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,2022-05
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,2022-02
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,2022-06
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941,2021-06
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232,2023-01


<a id='gestion' ></a>
## Gestion des produits et clients sans transaction<a href="#sommaire" style="font-size : 10px ; padding-left:10px">Retour Sommaire</a>

### Clients sans transactions

In [51]:
clientclient=customers['client_id'].unique().tolist()
clienttransac=transactions['client_id'].unique().tolist()

clientseul=[]
for client in clientclient:
    if client not in clienttransac : clientseul.append(client)
print (clientseul)

# Ces clients n'ont aucune transactions, on ne les gardera pas dans l'analyse
customers.loc[customers['client_id'].isin(clientseul)].head()

['c_8253', 'c_3789', 'c_4406', 'c_2706', 'c_3443', 'c_4447', 'c_3017', 'c_4086', 'c_6930', 'c_4358', 'c_8381', 'c_1223', 'c_6862', 'c_5245', 'c_5223', 'c_6735', 'c_862', 'c_7584', 'c_90', 'c_587', 'c_3526']


,client_id,sex,birth,age
801,c_8253,f,2001,22
2483,c_3789,f,1997,26
2734,c_4406,f,1998,25
2769,c_2706,f,1967,56
2851,c_3443,m,1959,64


In [52]:
prop = round(100*len(clientseul)/customers['client_id'].count(),2)
print(f'Ces clients sans transactions représentent {prop}% du nombre de clients total de customers.')

Ces clients sans transactions représentent 0.24% du nombre de clients total de customers.


### Produits sans transactions

In [53]:
productproduct=products['id_prod'].unique().tolist()
producttransac=transactions['id_prod'].unique().tolist()

produitseul=[]
for produit in productproduct:
    if produit not in producttransac : produitseul.append(produit)
print (produitseul)

# Ces clients n'ont aucune transactions, on ne les gardera pas dans l'analyse
products.loc[products['id_prod'].isin(produitseul)].head()

['0_1016', '0_1780', '0_1062', '0_1119', '0_1014', '1_0', '0_1318', '0_1800', '0_1645', '0_322', '0_1620', '0_1025', '2_87', '1_394', '2_72', '0_310', '0_1624', '2_86', '0_299', '0_510', '0_2308']


,id_prod,price,categ
184,0_1016,35.06,0
279,0_1780,1.67,0
737,0_1062,20.08,0
794,0_1119,2.99,0
811,0_1014,1.15,0


In [54]:
prop2 = round(100*len(produitseul)/products['id_prod'].count(),2)
print(f'Ces produits sans transactions représentent {prop2}% du nombre de produits total de products.')

Ces produits sans transactions représentent 0.64% du nombre de produits total de products.


<a id='jointure' ></a>
## Jointure<a href="#sommaire" style="font-size : 10px ; padding-left:10px">Retour Sommaire</a>

In [55]:
transactions.head()

,id_prod,date,session_id,client_id,mois
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,2022-05
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,2022-02
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,2022-06
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941,2021-06
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232,2023-01


In [56]:
products.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


In [57]:
customers.head()

,client_id,sex,birth,age
0,c_4410,f,1967,56
1,c_7839,f,1975,48
2,c_1699,f,1984,39
3,c_5961,f,1962,61
4,c_5320,m,1943,80


In [58]:
inter = pd.merge(transactions, products, on='id_prod', how='left')
inter.head()

,id_prod,date,session_id,client_id,mois,price,categ
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,2022-05,4.18,0
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,2022-02,15.99,1
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,2022-06,7.99,0
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941,2021-06,69.99,2
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232,2023-01,4.99,0


In [59]:
df_global = pd.merge(inter, customers, on='client_id', how='left')
df_global.head()

,id_prod,date,session_id,client_id,mois,price,categ,sex,birth,age
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,2022-05,4.18,0,f,1986,37
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,2022-02,15.99,1,m,1988,35
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,2022-06,7.99,0,f,1968,55
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941,2021-06,69.99,2,m,2000,23
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232,2023-01,4.99,0,m,1980,43


### Exploration

In [60]:
df_global.shape

(679332, 10)

In [61]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679332 entries, 0 to 679331
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     679332 non-null  object        
 1   date        679332 non-null  datetime64[ns]
 2   session_id  679332 non-null  object        
 3   client_id   679332 non-null  object        
 4   mois        679332 non-null  object        
 5   price       679111 non-null  float64       
 6   categ       679111 non-null  category      
 7   sex         679332 non-null  category      
 8   birth       679332 non-null  int64         
 9   age         679332 non-null  int64         
dtypes: category(2), datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 47.9+ MB


In [62]:
df_global.describe()

,price,birth,age
count,679111.000000,679332.000000,679332.000000
mean,17.454773,1977.811139,45.188861
std,18.328998,13.574553,13.574553
min,0.620000,1929.000000,19.000000
25%,8.870000,1970.000000,36.000000
50%,13.990000,1980.000000,43.000000
75%,18.990000,1987.000000,53.000000
max,300.000000,2004.000000,94.000000


### Traitement des valeurs manquantes

In [63]:
df_global.isna().sum()

id_prod         0
date            0
session_id      0
client_id       0
mois            0
price         221
categ         221
sex             0
birth           0
age             0
dtype: int64

In [64]:
# les 221 lignes où il y a des valeurs manquantes pour price et categ (les colonnes issues de products)
df_global[df_global.isna().any(axis=1)]

,id_prod,date,session_id,client_id,mois,price,categ,sex,birth,age
2633,0_2245,2022-09-23 07:22:38.636773,s_272266,c_4746,2022-09,NaN,NaN,m,1940,83
10103,0_2245,2022-07-23 09:24:14.133889,s_242482,c_6713,2022-07,NaN,NaN,f,1963,60
11723,0_2245,2022-12-03 03:26:35.696673,s_306338,c_5108,2022-12,NaN,NaN,m,1978,45
15670,0_2245,2021-08-16 11:33:25.481411,s_76493,c_1391,2021-08,NaN,NaN,m,1991,32
16372,0_2245,2022-07-16 05:53:01.627491,s_239078,c_7954,2022-07,NaN,NaN,m,1973,50
...,...,...,...,...,...,...,...,...,...,...
669533,0_2245,2021-08-25 09:06:03.504061,s_80395,c_131,2021-08,NaN,NaN,m,1981,42
670484,0_2245,2022-03-06 19:59:19.462288,s_175311,c_4167,2022-03,NaN,NaN,f,1979,44
671088,0_2245,2022-05-16 11:35:20.319501,s_209381,c_4453,2022-05,NaN,NaN,m,1981,42
675480,0_2245,2022-02-11 09:05:43.952857,s_163405,c_1098,2022-02,NaN,NaN,m,1986,37


In [65]:
df_global[df_global.isna().any(axis=1)]['id_prod'].unique()
# Toutes ces lignes concernent les transaction d'un seul produit qui a pour id_prod '0_2245'

array(['0_2245'], dtype=object)

In [66]:
products.loc[products['id_prod']=='0_2245', :]

,id_prod,price,categ


Aucun produit n'a cet id_prod dans products, cependant ce produit a été vendus 221 fois, ce qui n'est pas négligeable.

Les id_prod ont pour structure categ_numero, on sait donc que la catégorie de ce produit est 0.

On peut remplacer son prix par une moyenne, celle de sa catégorie par exemple.

### Vérification de notre théorie

In [67]:
# dataframe sans les lignes contenant les valeurs manquantes
df=df_global[~df_global.isna().any(axis=1)]

In [68]:
A=np.array(df['id_prod'])
B=[]
for i in A:
    B.append(i[0])
# B est la liste contenant le 1er caractère de id_prod, on va vérifier que ce terme est bien égal à categ

In [69]:
C=np.array(df['categ'])
D=list(C)
E=[]
for j in D:
    k=str(int(j))
    E.append(k)
# E est la liste contenant les valeurs de categ, dans le même type que B

In [70]:
# On compare les 2 listes 
if B==E: 
    print(True)
else :
    print(False)

True


Le résultat est True, on peut conclure que le 1er caractère des id_prod est égal à la catégorie.

### Imputation par la moyenne

In [71]:
df.groupby('categ')[['price']].mean()

,price
categ,
0,10.638188
1,20.485730
2,76.207412


La moyenne des prix par catégorie est significativement différente, on va donc modifié le prix de notre référence manquante par la moyenne de sa catégorie en terme de ventes et lui assigner sa catégorie dans categ.

In [72]:
# Remplacement du prix par 10.64, la moyenne de sa catégorie
df_global.loc[df_global['id_prod']=='0_2245', 'price']=round(df.groupby('categ')['price'].mean()[0],2)
# Assignement de sa catégorie à 0
df_global.loc[df_global['id_prod']=='0_2245', 'categ']=0

In [73]:
# les types ont été assignés et il n'a a plus de valeurs manquantes
df_global.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679332 entries, 0 to 679331
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     679332 non-null  object        
 1   date        679332 non-null  datetime64[ns]
 2   session_id  679332 non-null  object        
 3   client_id   679332 non-null  object        
 4   mois        679332 non-null  object        
 5   price       679332 non-null  float64       
 6   categ       679332 non-null  category      
 7   sex         679332 non-null  category      
 8   birth       679332 non-null  int64         
 9   age         679332 non-null  int64         
dtypes: category(2), datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 47.9+ MB


In [74]:
df_global.describe(include='all', datetime_is_numeric=True)

,id_prod,date,session_id,client_id,mois,price,categ,sex,birth,age
count,679332,679332,679332,679332,679332,679332.000000,679332.0,679332,679332.000000,679332.000000
unique,3266,NaN,342315,8600,24,NaN,3.0,2,NaN,NaN
top,1_369,NaN,s_118668,c_1609,2021-09,NaN,0.0,m,NaN,NaN
freq,2252,NaN,14,25488,33326,NaN,415680.0,340930,NaN,NaN
mean,NaN,2022-03-03 15:13:19.307389696,NaN,NaN,NaN,17.452557,NaN,NaN,1977.811139,45.188861
min,NaN,2021-03-01 00:01:07.843138,NaN,NaN,NaN,0.620000,NaN,NaN,1929.000000,19.000000
25%,NaN,2021-09-08 09:14:25.055994368,NaN,NaN,NaN,8.870000,NaN,NaN,1970.000000,36.000000
50%,NaN,2022-03-03 07:50:20.817730560,NaN,NaN,NaN,13.990000,NaN,NaN,1980.000000,43.000000
75%,NaN,2022-08-30 23:57:08.555173888,NaN,NaN,NaN,18.990000,NaN,NaN,1987.000000,53.000000
max,NaN,2023-02-28 23:58:30.792755,NaN,NaN,NaN,300.000000,NaN,NaN,2004.000000,94.000000


In [75]:
df_global.head()

,id_prod,date,session_id,client_id,mois,price,categ,sex,birth,age
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,2022-05,4.18,0,f,1986,37
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,2022-02,15.99,1,m,1988,35
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,2022-06,7.99,0,f,1968,55
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941,2021-06,69.99,2,m,2000,23
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232,2023-01,4.99,0,m,1980,43


Mon fichier de préparation est terminé, je vais l'exporter dans un nouveau fichier csv puis commencer mon analyse.

<a id='export' ></a>
## Export <a href="#sommaire" style="font-size : 10px ; padding-left:10px">Retour Sommaire</a>

In [76]:
df_global.to_csv(folder+'global.csv', index=False)

In [77]:
customers.to_csv(folder+'customers_2.csv', index=False)

In [78]:
products.to_csv(folder+'products_2.csv', index=False)

In [79]:
df_global

,id_prod,date,session_id,client_id,mois,price,categ,sex,birth,age
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,2022-05,4.18,0,f,1986,37
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,2022-02,15.99,1,m,1988,35
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,2022-06,7.99,0,f,1968,55
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941,2021-06,69.99,2,m,2000,23
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232,2023-01,4.99,0,m,1980,43
...,...,...,...,...,...,...,...,...,...,...
679327,0_1551,2022-01-15 13:05:06.246925,s_150195,c_8489,2022-01,12.99,0,f,1951,72
679328,1_639,2022-03-19 16:03:23.429229,s_181434,c_4370,2022-03,10.99,1,f,1977,46
679329,0_1425,2022-12-20 04:33:37.584749,s_314704,c_304,2022-12,12.99,0,f,1988,35
679330,0_1994,2021-07-16 20:36:35.350579,s_63204,c_2227,2021-07,4.98,0,m,1986,37
